In [1]:
import pandas as pd
import numpy as np

import requests
import csv

import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim 
from geopy.exc import GeocoderTimedOut
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [2]:
sydney_suburbs = pd.read_csv('sydney_suburbs.csv')
sydney_suburbs.dropna(inplace=True)
sydney_suburbs.reset_index(inplace=True)
sydney_suburbs.drop(['index'],inplace=True,axis=1)
sydney_suburbs

,Suburb,Latitude,Longitude
0,Abbotsbury,-33.869285,150.866703
1,Abbotsford,-33.850553,151.129759
2,Acacia Gardens,-33.732459,150.912532
3,Agnes Banks,-33.614508,150.711448
4,Airds,-34.090000,150.826111
...,...,...,...
673,Yarramundi,-33.631817,150.667996
674,Yarrawarrah,-34.056681,151.030888
675,Yennora,-33.864629,150.971150
676,Yowie Bay,-34.050278,151.103333


In [3]:
sydney_stations=pd.read_csv("sydney_stations.csv")
sydney_stations.dropna(inplace=True)
sydney_stations.reset_index(inplace=True)
sydney_stations.drop(['index'],inplace=True,axis=1)
sydney_stations

,Station,Latitude,Longitude
0,Allawah Railway Station,-33.969629,151.114285
1,Arncliffe Railway Station,-33.938236,151.145508
2,Artarmon Railway Station,-33.808955,151.185309
3,Ashfield Railway Station,-33.889478,151.127412
4,Asquith Railway Station,-33.687193,151.111026
...,...,...,...
162,Wollstonecraft Railway Station,-33.833204,151.195927
163,Woolooware Railway Station,-34.044841,151.140172
164,Wynyard Railway Station,-33.865990,151.205810
165,Yagoona Railway Station,-33.903791,151.017668


In [4]:
#Obtain the latitude and longitude of Sydney
address = 'Sydney, Australia'
geolocator = Nominatim(user_agent="Sydney_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney are -33.8548157, 151.2164539.


In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=3000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):

            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
                                                                                                                                       CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            response = requests.get(url).json()
            results = response["response"]['venues']
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [12]:
# #soujanya927
# CLIENT_ID = "XIGGCC3DSUICPZOBTPEVSLHEDW5JXYW4MCHZA2SG5N4TJUUG"
# CLIENT_SECRET = "JFT241ZRHU3DU2XL2YBOPYXQROEK332GWIOYZ4BHTO5DQMBJ"

# #soujanya.kodavalla
CLIENT_ID = "MW0HNMAAVZ2AHN2GJQVRURBGN5HEORAR0EVEYMKI10WYYNNM"
CLIENT_SECRET = "DQ1ZEWFMCBKBC0311AJQE1T3HNSTQRP1RDVWOFZCVYLS20DB"

# #hisurya08
# CLIENT_ID = "DURBJMCWJUOTAR4T1N1RGVTO5EUZ5IW5453SC2QRXFMPAXPP"
# CLIENT_SECRET = "11ARICNQL51RPTQGVT2ECHMIRUDUB32R2ZMRP45MWDBYRFBQ"

# #2015mailtossk
# CLIENT_ID = "J4QZZX2MXUW3STNBKHXP2GRM3PAJTGLCQKENFIFRVBPCGPB2"
# CLIENT_SECRET = "KETVPCI5DOD25H3DKNIIAWUFVTWXGR0PDGIS1XPMDOTRK3LZ"

# #foursquaressk
# CLIENT_ID = "400PANWAC5VG53ANSL54PY2AKADJFCPKJR0WHX11NGDH0AKB"
# CLIENT_SECRET = "T5AHL1T1G2Q0A0GDKNCEKEPLEHXXQTAHH5KDWMZPIFLZHQPI"

VERSION = '20180604'
LIMIT = 20

In [13]:
top_food_places_original = getNearbyVenues(names=sydney_suburbs['Suburb'], latitudes=sydney_suburbs['Latitude'], longitudes=sydney_suburbs['Longitude'], 
                                  radius=2500, categoryIds='4d4b7105d754a06374d81259')
#categoryIds is 4d4b7105d754a06374d81259 as it returns all the food places
top_food_places_original

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbotsbury,-33.869285,150.866703,Orchid House,-33.876449,150.877346,Thai Restaurant
1,Abbotsbury,-33.869285,150.866703,McDonald's,-33.888236,150.881752,Fast Food Restaurant
2,Abbotsbury,-33.869285,150.866703,Thai Orchard House,-33.877573,150.882771,Thai Restaurant
3,Abbotsbury,-33.869285,150.866703,Pisa Cakes,-33.876193,150.876837,Dessert Shop
4,Abbotsbury,-33.869285,150.866703,Giglio's Fresh Chickens,-33.886153,150.870167,Fried Chicken Joint
...,...,...,...,...,...,...,...
12078,Zetland,-33.907662,151.208218,Starbucks,-33.884330,151.199484,Coffee Shop
12079,Zetland,-33.907662,151.208218,Gelato Messina,-33.918210,151.203100,Ice Cream Shop
12080,Zetland,-33.907662,151.208218,The Grounds of Alexandria,-33.910774,151.194406,Café
12081,Zetland,-33.907662,151.208218,Little Evie,-33.892748,151.214291,Café


In [14]:
top_food_places_original.to_csv(r'top_food_places_initial.csv', index = False)
top_food_places= top_food_places_original
top_food_places

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbotsbury,-33.869285,150.866703,Orchid House,-33.876449,150.877346,Thai Restaurant
1,Abbotsbury,-33.869285,150.866703,McDonald's,-33.888236,150.881752,Fast Food Restaurant
2,Abbotsbury,-33.869285,150.866703,Thai Orchard House,-33.877573,150.882771,Thai Restaurant
3,Abbotsbury,-33.869285,150.866703,Pisa Cakes,-33.876193,150.876837,Dessert Shop
4,Abbotsbury,-33.869285,150.866703,Giglio's Fresh Chickens,-33.886153,150.870167,Fried Chicken Joint
...,...,...,...,...,...,...,...
12078,Zetland,-33.907662,151.208218,Starbucks,-33.884330,151.199484,Coffee Shop
12079,Zetland,-33.907662,151.208218,Gelato Messina,-33.918210,151.203100,Ice Cream Shop
12080,Zetland,-33.907662,151.208218,The Grounds of Alexandria,-33.910774,151.194406,Café
12081,Zetland,-33.907662,151.208218,Little Evie,-33.892748,151.214291,Café


In [15]:
venue_lats = top_food_places['Venue Latitude']
venue_longs=top_food_places['Venue Longitude']
station_lats=sydney_stations['Latitude']
station_longs=sydney_stations['Longitude']
stations=sydney_stations['Station']

a,b=top_food_places.shape
c,d=sydney_stations.shape

#changed due to issues with nan data
#station_lats[72]=0
#station_longs[72]=0

In [16]:
from geopy.distance import great_circle

nearest_station=[None] * a
distance= [None] * a

i=0
for i in range(a):
    venue=(venue_lats[i],venue_longs[i])
    dist=float('inf')
    j=0
    for j in range(c):
        station=(station_lats[j],station_longs[j])
        temp=great_circle(venue,station).km
        if(temp<dist):
            dist = temp
            distance[i] = temp
            nearest_station[i]=stations[j]

In [17]:
#Combine the calculations obtained with the dataset previously used
top_food_places.insert(loc=7,column="Nearest Station",value=nearest_station) 
top_food_places.insert(loc=8,column="Distance to Nearest Station in km",value=distance) 
top_food_places

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Nearest Station,Distance to Nearest Station in km
0,Abbotsbury,-33.869285,150.866703,Orchid House,-33.876449,150.877346,Thai Restaurant,Cabramatta Railway Station,5.805052
1,Abbotsbury,-33.869285,150.866703,McDonald's,-33.888236,150.881752,Fast Food Restaurant,Cabramatta Railway Station,4.991601
2,Abbotsbury,-33.869285,150.866703,Thai Orchard House,-33.877573,150.882771,Thai Restaurant,Cabramatta Railway Station,5.298811
3,Abbotsbury,-33.869285,150.866703,Pisa Cakes,-33.876193,150.876837,Dessert Shop,Cabramatta Railway Station,5.859627
4,Abbotsbury,-33.869285,150.866703,Giglio's Fresh Chickens,-33.886153,150.870167,Fried Chicken Joint,Cabramatta Railway Station,6.085688
...,...,...,...,...,...,...,...,...,...
12078,Zetland,-33.907662,151.208218,Starbucks,-33.884330,151.199484,Coffee Shop,Redfern Railway Station,1.062016
12079,Zetland,-33.907662,151.208218,Gelato Messina,-33.918210,151.203100,Ice Cream Shop,Green Square Railway Station,1.336991
12080,Zetland,-33.907662,151.208218,The Grounds of Alexandria,-33.910774,151.194406,Café,Green Square Railway Station,0.898916
12081,Zetland,-33.907662,151.208218,Little Evie,-33.892748,151.214291,Café,Redfern Railway Station,0.948046


In [18]:
top_food_places.to_csv(r'top_food_places.csv', index = False)